In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import os
import torch
import warnings
from Module.sam import sam_model_registry, SamPredictor
from Module.sam.utils.onnx import SamOnnxModel

from Module.models.timm_image_encoder import TimmImageEncoder
from mobile_sam.modeling.sam import Sam

C:\Users\danusys16\anaconda3\envs\sam\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\danusys16\anaconda3\envs\sam\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
C:\Users\danusys16\anaconda3\envs\sam\lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
D:\015_segmentation\Custom_SAM\Module\sam\modeling\tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_5m_224 in registry with Module.sam.modeling.tiny_vit_sam.

ModuleNotFoundError: No module named 'mobile_sam'

## Model Export

### Common Settomg

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
opset=20

### Encoder

In [4]:
enc_type = 'resnet18'
enc_checkpoint = "../runs/241120_vit_t_to_resnet18_DANU_WS_v2_01/tiny_vit_best.pth"

In [5]:
## Model load
sam_enc = TimmImageEncoder('resnet18', pretrained=True)
sam_enc.load_state_dict(torch.load(enc_checkpoint)["model"])

sam_enc.to(device)
sam_enc.eval();

C:\Users\danusys16\AppData\Local\Temp\ipykernel_21208\1401888577.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sam_enc.load_state_dict(torch.load(enc_checkpoint)["mode

In [6]:
input_size=1024
dummy_input = torch.randn(1, 3, input_size, input_size).to(device)

In [8]:
output_enc_path = "../Weights/onnx/water_seg_resnet18_encoder.onnx" ##enc_checkpoint.replace(".pth", "_encoder.onnx")

torch.onnx.export(
            sam_enc,
            (dummy_input,),
            output_enc_path,
            input_names=["image"],
            output_names=["image_embeddings"],
            opset_version=opset
        )

### Decoder

In [9]:
dec_model_type = 'vit_b'
dec_checkpoint = '../runs/241115_SAM_ViT_b_dataset_V2_ft_v1/best.pth'

sam = sam_model_registry[dec_model_type](checkpoint=dec_checkpoint)

D:\015_segmentation\Custom_SAM\Module\sam\build_sam.py:154: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


In [10]:
onnx_model = SamOnnxModel(
    model=sam, 
    return_single_mask=True,
    use_stability_score=False,
    return_extra_metrics=False
)

In [17]:
output_dec_path = "../Weights/onnx/water_seg_decoder.onnx" ##dec_checkpoint.replace("best.pth", "water_seg_vit_b_decoder.onnx")

In [18]:
dynamic_axes = {
        "point_coords": {1: "num_points"},
        "point_labels": {1: "num_points"},
    }

embed_dim = sam.prompt_encoder.embed_dim
embed_size = sam.prompt_encoder.image_embedding_size
mask_input_size = [4 * x for x in embed_size]
dummy_inputs = {
        "image_embeddings": torch.randn(1, embed_dim, *embed_size, dtype=torch.float),
        "point_coords": torch.randint(low=0, high=1024, size=(1, 5, 2), dtype=torch.float),
        "point_labels": torch.randint(low=0, high=4, size=(1, 5), dtype=torch.float),
        "mask_input": torch.randn(1, 1, *mask_input_size, dtype=torch.float),
        "has_mask_input": torch.tensor([1], dtype=torch.float),
        "orig_im_size": torch.tensor([1500, 2250], dtype=torch.int32),
    }

In [19]:
_ = onnx_model(**dummy_inputs)

In [20]:
output_names = ["masks", "iou_predictions", "low_res_masks"]

In [21]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=torch.jit.TracerWarning)
    warnings.filterwarnings("ignore", category=UserWarning)
    with open(output_dec_path, "wb") as f:
        print(f"Exporting onnx model to {output_dec_path}...")
        torch.onnx.export(
            onnx_model,
            tuple(dummy_inputs.values()),
            f,
            input_names=list(dummy_inputs.keys()),
            output_names=["masks", "iou_predictions", "low_res_masks"],
            dynamic_axes=dynamic_axes,
            export_params=True,
            verbose=False,
            opset_version=20,
            do_constant_folding=True, 
            )

Exporting onnx model to ../Weights/onnx/water_seg_decoder.onnx...


In [22]:
list(dummy_inputs.keys())

['image_embeddings',
 'point_coords',
 'point_labels',
 'mask_input',
 'has_mask_input',
 'orig_im_size']